In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
from tensorflow.keras.utils import plot_model

In [5]:
import json

with open(r'C:\Users\91630\Desktop\Intent.json') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])
df.head()

,intent,text,responses
0,greetings,"[hello, hey, hi, good day, greetings, what's u...","[hello, hey!, what can i do for you?]"
1,goodbye,"[cya, see you later, goodbye, have a good day,...","[have a nice day, goodbye]"
2,age,"[how old, how old are you?, what is your age, ...","[I get reborn after every compilation, hey!, m..."
3,name,"[what is your name, what should i call you, wh...","[you can call me Medbot!, i am Medbot!, i am M..."
4,common cold symptoms,"[Runny or stuffy nose, Sore throat, Cough, Con...",[It seems that you are suffering from common c...


In [7]:
dic = {"intent":[],"text":[], "responses":[]}
for i in range (len(df)):
    ptrns = df[df.index==i]['text'].values[0]
    rspns = df[df.index==i]['responses'].values[0]
    tag = df[df.index==i]['intent'].values[0]
    for j in range(len(ptrns)):
        dic['intent'].append(tag)
        dic['text'].append(ptrns[j])
        dic['responses'].append(rspns)
df = pd.DataFrame.from_dict(dic)
df

,intent,text,responses
0,greetings,hello,"[hello, hey!, what can i do for you?]"
1,greetings,hey,"[hello, hey!, what can i do for you?]"
2,greetings,hi,"[hello, hey!, what can i do for you?]"
3,greetings,good day,"[hello, hey!, what can i do for you?]"
4,greetings,greetings,"[hello, hey!, what can i do for you?]"
...,...,...,...
343,mindfulness_meditation,Mindful eating meditation,[Mindfulness meditation involves being fully p...
344,aging_gracefully,Aging gracefully tips,"[Aging gracefully involves staying active, soc..."
345,aging_gracefully,Maintaining mental agility in old age,"[Aging gracefully involves staying active, soc..."
346,aging_gracefully,Embracing the aging process,"[Aging gracefully involves staying active, soc..."


In [9]:
df['intent'].unique()

array(['greetings', 'goodbye', 'age', 'name', 'common cold symptoms',
       'fever symptoms', 'Diabetes symptoms', 'Depression symptoms',
       'Asthma symptoms', 'common cold prevention', 'fever prevention',
       'diabetes prevention', 'depression prevention',
       'asthma prevention', 'Consultation', 'exercise_recommendation',
       'nutrition_advice', 'sleep_health', 'holistic_health',
       'travel_health', 'first_aid', 'aging_health', 'sexual_health',
       'home_remedies', 'stress_management', 'allergies', 'pregnancy',
       'child_health', 'elderly_care', 'mental_health_support',
       'rehabilitation', 'substance_abuse', 'chronic_conditions',
       'dental_care', 'symptoms', 'emergency', 'medication',
       'appointment', 'insurance', 'mental_health', 'diet_nutrition',
       'exercise', 'vaccination', 'insurance_claim', 'self_care_tips',
       'nutrition_labels', 'sleep_disorders', 'immune_system',
       'exercise_injuries', 'weight_management', 'digital_health'

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df['text'])
tokenizer.get_config()

{'num_words': None,
 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
 'lower': True,
 'split': ' ',
 'char_level': False,
 'oov_token': None,
 'document_count': 348,
 'word_counts': '{"hello": 1, "hey": 1, "hi": 1, "good": 2, "day": 2, "greetings": 1, "what\'s": 3, "up": 2, "how": 17, "is": 6, "it": 1, "going": 1, "cya": 1, "see": 2, "you": 8, "later": 2, "goodbye": 1, "have": 3, "a": 20, "bye": 1, "cao": 1, "ya": 1, "old": 5, "are": 4, "what": 17, "your": 4, "age": 4, "name": 3, "should": 8, "i": 31, "call": 2, "who": 2, "can": 16, "tell": 6, "me": 12, "runny": 1, "or": 9, "stuffy": 1, "nose": 1, "sore": 2, "throat": 2, "cough": 1, "congestion": 1, "slight": 1, "body": 7, "aches": 2, "mild": 1, "headache": 2, "sneezing": 1, "low": 2, "grade": 1, "fever": 5, "generally": 1, "feeling": 1, "unwell": 1, "malaise": 1, "sweating": 1, "chills": 1, "and": 16, "shivering": 1, "muscle": 2, "loss": 4, "of": 19, "appetite": 2, "irritability": 1, "dehydration": 1, "general": 1, "weakness": 1, "

In [17]:
vocab_size = len(tokenizer.word_index)
print('number of unique words = ', vocab_size)

number of unique words =  533


In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

ptrn2seq = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['intent'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))

X shape =  (348, 13)
y shape =  (348,)
num of classes =  80


In [19]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
from tensorflow.keras.utils import plot_model

model = Sequential()
# Initialize the Sequential model

# Add Input layer
model.add(Input(shape=(X.shape[1],)))

# Add Embedding layer
model.add(Embedding(input_dim=vocab_size + 1, output_dim=100, mask_zero=True))

# Add LSTM and LayerNormalization layers
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())

model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())

model.add(LSTM(32))
model.add(LayerNormalization())

# Add Dense, LayerNormalization, and Dropout layers
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))

model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))

# Output layer with the number of unique labels in y
model.add(Dense(len(np.unique(y)), activation="softmax"))

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 13, 100)        │        53,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 13, 32)         │        17,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 13, 32)         │            64 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 13, 32)         │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (None, 13, 32)         │            64 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ (None, 32)             │            64 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_3           │ (None, 128)            │           256 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_4           │ (None, 128)            │           256 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 80)             │        10,320 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,824 (464.16 KB)

 Trainable params: 118,824 (464.16 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [22]:
model_history = model.fit(x=X,
                          y=y,
                          batch_size=10,
                          callbacks=[tensorflow.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)],
                          epochs=50)

Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.0097 - loss: 5.1390
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1122 - loss: 3.9209
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2577 - loss: 3.1691
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3597 - loss: 2.7553
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4952 - loss: 2.2955
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5132 - loss: 1.9691
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6879 - loss: 1.5660
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6865 - loss: 1.3536
Epoch 9/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7909 - loss: 1.1371
Epoch 10/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8544 - loss: 0.8276
Epoch 11/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8655 - loss: 0.7760
Epoch 12/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy:

In [23]:
import pickle

# Save the model
model.save('medical_healthcare_chatbot_model.h5')

# Save the tokenizer
with open('healthcare_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the label encoder
with open('healthcare_label_encoder.pickle', 'wb') as handle:
    pickle.dump(lbl_enc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
with open('medical_tokenizer.json', 'w') as f:
    json.dump(tokenizer.to_json(), f)
with open('medical_label_encoder.json', 'w') as f:
    json.dump(lbl_enc.classes_.tolist(), f)

In [ ]:
import re
import random

In [ ]:
def generate_answer(pattern): 
    text = []
    txt = re.sub('[^a-zA-Z\']', ' ', pattern)
    txt = txt.lower()
    txt = txt.split()
    txt = " ".join(txt)
    text.append(txt)
        
    x_test = tokenizer.texts_to_sequences(text)
    x_test = np.array(x_test).squeeze()
    x_test = pad_sequences([x_test], padding='post', maxlen=X.shape[1])
    y_pred = model.predict(x_test)
    y_pred = y_pred.argmax()
    tag = lbl_enc.inverse_transform([y_pred])[0]
    responses = df[df['intent'] == tag]['responses'].values[0]

    print("you: {}".format(pattern))
    print("Chitti The Chatbot: {}".format(random.choice(responses)))

In [31]:
generate_answer('Slight body aches or a mild headache')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
you: Slight body aches or a mild headache
Chitti The Chatbot: It seems that you are suffering from common cold
